In [324]:
from sklearn import datasets
import pandas as pd
import numpy as np
import time

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from tensorflow.keras import backend as K

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.metrics import f1_score, plot_confusion_matrix, confusion_matrix
from sklearn.compose import ColumnTransformer

from imblearn.pipeline import Pipeline as imbPipeline

from category_encoders import TargetEncoder

import warnings
warnings.filterwarnings('ignore')

In [226]:
X_train = pd.read_csv('Data/X_train.csv', index_col='building_id')
X_test = pd.read_csv('Data/X_test.csv', index_col='building_id')
y_train = pd.read_csv('Data/y_train.csv', index_col='building_id')
y_test = pd.read_csv('Data/y_test.csv', index_col='building_id')

### Reshape y_train for the Keras model

I need y_train in the shape (len(y_train), 3).  I also need to re-encode y so that 1, 2, 3 goes to 0, 1, 2, so that I can use np.utils.to_categorical.  When I do my predictions, I will reverse this encoding.

In [143]:
y_train.values.reshape(1, -1)

array([[2, 2, 2, ..., 2, 3, 2]])

In [165]:
y_train['target'] = y_train.damage_grade.apply(lambda x: x - 1)

In [178]:
dummy_y_train = np_utils.to_categorical(y_train.target)

In [179]:
dummy_y_train.shape

(195450, 3)

### Column Transformer

I will be doing the ususal transformations:  dropping unimportant columns, log-transforming and scaling the integer columns, and target encoding the categorical columns.

In [217]:
binary_cols = []
for col in X_train.columns:
    if col.startswith('has'):
        binary_cols.append(col)

cat_cols = list(X_train.select_dtypes(include='object').columns)

integer_cols = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']

geo_cols = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']

all_cols = geo_cols + cat_cols + integer_cols + binary_cols

In [218]:
binary_cols_dropped = binary_cols.copy()
for col in binary_cols_dropped:
    if col.startswith('has_secondary'):
        binary_cols_dropped.remove(col)
binary_cols_dropped.append('has_secondary_use')

cat_cols_dropped = cat_cols.copy()
cat_cols_dropped.remove('legal_ownership_status')
cat_cols_dropped.remove('plan_configuration')

In [219]:
def log_transform(x):
    return np.log(x+1)

In [224]:
function_transformer = FunctionTransformer(log_transform)

ohe_pipe = imbPipeline([('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
integer_pipe = imbPipeline([
    ('function', function_transformer),
    ('ss', StandardScaler())
])
target_pipe = imbPipeline([('target', TargetEncoder(cols=geo_cols))])

transformer = ColumnTransformer([
    ('binary', 'passthrough', binary_cols_dropped),
    ('categorical', ohe_pipe, cat_cols_dropped),
    ('geo', target_pipe, geo_cols),
    ('integer', integer_pipe, integer_cols),
])

In [230]:
transformed_X_train = transformer.fit_transform(X_train, y_train)

In [232]:
transformed_X_train.shape

(195450, 49)

The final X_train has 49 features.

### Build the neural network

In [318]:
def keras0():    
    keras0 = Sequential()
    keras0.add(Dense(64, input_dim=49, activation='relu'))
    keras0.add(Dense(32, activation='relu'))
    keras0.add(Dense(16, activation='relu'))
    keras0.add(Dense(3, activation='softmax'))
    
    keras0.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return keras0

In [321]:
start = time.time()
keras0().fit(transformed_X_train, dummy_y_train, epochs=50, batch_size=1000)
end = time.time()
print(f'Runtime: {end-start} seconds')

Epoch 1/50
196/196 [==============================] - 0s 2ms/step - loss: 0.7295 - accuracy: 0.6638
Epoch 2/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5859 - accuracy: 0.7430
Epoch 3/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5782 - accuracy: 0.7451
Epoch 4/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5727 - accuracy: 0.7466
Epoch 5/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5693 - accuracy: 0.7478
Epoch 6/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5674 - accuracy: 0.7477
Epoch 7/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5666 - accuracy: 0.7479
Epoch 8/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5663 - accuracy: 0.7482
Epoch 9/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5655 - accuracy: 0.7482
Epoch 10/50
196/196 [==============================] - 0s 2ms/step - loss: 0.5644 - accuracy: 0.7493

### Create a cross validation procedure

In [319]:
keras0_estimator = KerasClassifier(build_fn=keras0, epochs=50, batch_size=1000, verbose=1)

In [322]:
start = time.time()
f1_micro_keras0 = cross_val_score(keras0_estimator, transformed_X_train, dummy_y_train)
end = time.time()
print(f'Run time: {end-start}')
run_time_keras0 = end-start

Epoch 1/50
157/157 [==============================] - 0s 2ms/step - loss: 0.7578 - accuracy: 0.6414
Epoch 2/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5846 - accuracy: 0.7423
Epoch 3/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5746 - accuracy: 0.7454
Epoch 4/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5718 - accuracy: 0.7461
Epoch 5/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5698 - accuracy: 0.7470
Epoch 6/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5680 - accuracy: 0.7472
Epoch 7/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5671 - accuracy: 0.7487
Epoch 8/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5652 - accuracy: 0.7489
Epoch 9/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5646 - accuracy: 0.7493
Epoch 10/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5643 - accuracy: 0.7491

157/157 [==============================] - 1s 3ms/step - loss: 0.5557 - accuracy: 0.7521
Epoch 32/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5549 - accuracy: 0.7523
Epoch 33/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5553 - accuracy: 0.7521
Epoch 34/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5548 - accuracy: 0.7521
Epoch 35/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5545 - accuracy: 0.7528
Epoch 36/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5549 - accuracy: 0.7519
Epoch 37/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5542 - accuracy: 0.7527
Epoch 38/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5539 - accuracy: 0.7528
Epoch 39/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5533 - accuracy: 0.7530
Epoch 40/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5550 - accuracy: 0.7529
Ep

157/157 [==============================] - 0s 2ms/step - loss: 0.5653 - accuracy: 0.7490
Epoch 11/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5646 - accuracy: 0.7488
Epoch 12/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5641 - accuracy: 0.7487
Epoch 13/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5634 - accuracy: 0.7490
Epoch 14/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5636 - accuracy: 0.7490
Epoch 15/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.7499
Epoch 16/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5622 - accuracy: 0.7507
Epoch 17/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5621 - accuracy: 0.7501: 0s - loss: 0.5631 - ac
Epoch 18/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5617 - accuracy: 0.7503
Epoch 19/50
157/157 [==============================] - 0s 3ms/step - loss: 0.56

157/157 [==============================] - 0s 3ms/step - loss: 0.5552 - accuracy: 0.7513
Epoch 40/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5555 - accuracy: 0.7515
Epoch 41/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5553 - accuracy: 0.7511
Epoch 42/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5547 - accuracy: 0.7511
Epoch 43/50
157/157 [==============================] - 1s 3ms/step - loss: 0.5540 - accuracy: 0.7520
Epoch 44/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5539 - accuracy: 0.7514
Epoch 45/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5538 - accuracy: 0.7514
Epoch 46/50
157/157 [==============================] - 0s 2ms/step - loss: 0.5545 - accuracy: 0.7519
Epoch 47/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5537 - accuracy: 0.7522
Epoch 48/50
157/157 [==============================] - 0s 3ms/step - loss: 0.5529 - accuracy: 0.7519
Ep

### Transform X_test

In [281]:
transformed_X_test = transformer.transform(X_test)

### Make predictions on X_test and score the results

After predicting classes, I will reverse transform the predictions so that 0, 1, 2 is mapped to 1, 2, 3 (the original target.)

In [304]:
y_pred = model.predict_classes(transformed_X_test)

In [305]:
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_adj'], index=X_test.index)

In [306]:
y_pred_df['y_pred'] = y_pred_df['y_pred_adj'].apply(lambda x: x + 1)

In [307]:
keras_score = f1_score(y_test, y_pred_df.y_pred, average='micro')
keras_score

0.7314239228868321